<h1> โปรแกรมการอัปโหลดภาพ


In [ ]:
from PyQt5.QtWidgets import QFileDialog
import cv2
def importfile(self):
        Filename = QFileDialog.getOpenFileName(self,'Open Image','',
        'ImageFiles (*.png *.jpg *.jpeg)')
        img = cv2.imread(Filename[0])
        img = cv2.resize(img ,(224,224))

<h1> โปรแกรมการประมวลผลภาพ

In [ ]:
import numpy as np
import cv2
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
rimg = np.array(img)
rimg = rimg.astype('float32')
rimg /= 255
rimg = np.reshape(rimg ,(1,224,224,1))

<h1>โปรแกรมการจำแนกกลุ่มของภาพด้วยเทคนิคโครงข่ายประสาทเทียมแบบคอนโวลูชัน

  - โค้ดการทำ Preprocess data ไว้สำหรับ Train โมเดล
ซึ่งเป็นส่วนของขั้นตอนดำเนินงาน อัปโหลดไฟล์ภาพและการประมวลผลภาพ

In [ ]:
TRAIN_DIR = ['data/PNEUMONIA','data/NORMAL','data/COVID-19']
TEST_DIR = ['test/PNEUMONIA', test/NORMAL', 'test/COVID-19']
def create_train_data(DIR):
        rawImgs = []
        labels = []
        for imagePath in DIR:
            for item in tqdm(os.listdir(imagePath)):
               file = os.path.join(imagePath, item)
                  if file[-1] == 'g':
                    name = imagePath.split('/')[-1]
                    if name == 'PNEUMONIA':
                       labels.append([1,0,0])
                    elif name == 'NORMAL':
                       labels.append([0,1,0])
                    else :
                       labels.append([0,0,1])
                    img = cv2.imread(file)
                    img = cv2.resize(img ,(224,224))
                    img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
                    rawImgs.append(img)
          return rawImgs, labels

x_train,y_train = create_train_data(TRAIN_DIR)
x_test,y_test = create_train_data(TEST_DIR)
x_train = np.array(x_train)
y_train = np.array(y_train)
x_test = np.array(x_test)
y_test = np.array(y_test)
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

  - โค้ดการสร้างโมเดล

In [ ]:
model = tf.keras.Sequential([
       tf.keras.layers.Conv2D(64, (3,3), activation='relu', input_shape=(224, 224, 1)),
       tf.keras.layers.Conv2D(64,(3,3) , activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2, 2)),
       tf.keras.layers.Conv2D(128,(3,3) , activation='relu'),
       tf.keras.layers.Conv2D(128,(3,3) , activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2 )),
       tf.keras.layers.Conv2D(256,(3,3) , activation='relu'),
       tf.keras.layers.Conv2D(256,(3,3) , activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2 )),
       tf.keras.layers.Conv2D(512,(3,3) , activation='relu'),
       tf.keras.layers.Conv2D(512,(3,3) , activation='relu'),
       tf.keras.layers.MaxPooling2D(pool_size=(2,2 )),
       tf.keras.layers.Flatten(),
       tf.keras.layers.Dense(64,activation=’relu’),
       tf.keras.layers.Dense(32,activation=’relu’),
       tf.keras.layers.Dense(16,activation=’relu’),
       tf.keras.layers.Dense(8,activation=’relu’),
       tf.keras.layers.Dense(3,activation=’softmax’)])
    ])
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.0001), loss='categorical_crossentropy',metrics= ['accuracy'])
batch_size = 8
epochs = 50
history = model.fit(x_train,y_train ,batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test))
filepath='modelCovid.h5'
model.save(filepath)

  - โค้ดการโหลดโมเดล CNN ที่ผ่านการเรียนรู้มาใช้งาน

In [ ]:
path = 'modelCovid.h5'
model = load_model(path) 
img = cv2.imread(file)
img = cv2.resize(img ,(224,224)) 
img = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)    
rimg = np.array(img)
rimg = rimg.astype('float32')
rimg /= 255
rimg = np.reshape(rimg ,(1,224,224,1))
predict = model.predict(rimg)
label = ['PNEUMONIA','NORMAL','COVID-19']
result = label[np.argmax(predict)]
print(predict)
print('real:'+str(item))
print('predict:'+str(result))